In [ ]:
# Imports
import numpy as np

from keras import Sequential, Input
from keras.applications import VGG16
from keras.datasets import fashion_mnist
from tensorflow.image import resize
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.layers import Dense

In [ ]:
# Loads dataset and corrects the number of channels
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
print(f"Before resize { x_train.shape } and {x_test.shape }")

# Will do the following to git VGG-16 params:
# Adds axis on the last index of the list
# Repeats 3 times the last axis
# Resize images
print(x_train[0])

# Resizes train set
x_train = np.expand_dims(x_train, axis=-1)
x_train = np.repeat(x_train, 3, axis=-1)
x_train = resize(x_train, [32,32])

# Resizes test set
x_test = np.expand_dims(x_test, axis=-1)
x_test = np.repeat(x_test, 3, axis=-1)
x_test = resize(x_test, [32,32])


print(f"After resize { x_train.shape } and {x_test.shape }")

Before resize (60000, 28, 28) and (10000, 28, 28)
After resize (60000, 32, 32, 3) and (10000, 32, 32, 3)


In [ ]:
# Instantiates VGG16
vgg16 = VGG16(
    include_top=True,
    weights=None,
    input_shape=(32,32,3),
    classes=10,
)

vgg16.summary()

# Compiles the model to be user
vgg16.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=1e-3), metrics=['accuracy'])

# Trains the model
vgg16.fit(x_train, to_categorical(y_train), batch_size=128, shuffle=True, epochs=10, validation_data=(x_test, to_categorical(y_test)))

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_16 (InputLayer)       [(None, 32, 32, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 32, 32, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 32, 32, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 16, 16, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 16, 16, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 16, 16, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 8, 8, 128)         0     

In [ ]:
# Loads dataset again - Overrides for the new custom network
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

# Transform and normalize
num_pixels = x_train.shape[1] * x_train.shape[2]
x_train = x_train.reshape(x_train.shape[0], num_pixels).astype('float32')
x_train = x_train / 255

x_test = x_test.reshape(x_test.shape[0], num_pixels).astype('float32')
x_test = x_test / 255

# Creates custom network
custom = Sequential()
custom.add(Input(shape=num_pixels))
custom.add(Dense(num_pixels, kernel_initializer='normal', activation='relu'))
custom.add(Dense(1024, kernel_initializer='normal', activation='relu'))
custom.add(Dense(2048, kernel_initializer='normal', activation='relu'))
custom.add(Dense(2048, kernel_initializer='normal', activation='relu'))
custom.add(Dense(10, kernel_initializer='normal', activation='softmax'))
custom.summary()

# Compiles model
custom.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=1e-3), metrics=['accuracy'])

# Trains model
custom.fit(x_train, to_categorical(y_train), batch_size=128, shuffle=True, epochs=10, validation_data=(x_test, to_categorical(y_test)))

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_30 (Dense)            (None, 784)               615440    
                                                                 
 dense_31 (Dense)            (None, 1024)              803840    
                                                                 
 dense_32 (Dense)            (None, 2048)              2099200   
                                                                 
 dense_33 (Dense)            (None, 2048)              4196352   
                                                                 
 dense_34 (Dense)            (None, 10)                20490     
                                                                 
Total params: 7,735,322
Trainable params: 7,735,322
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
469/469 [========================